1. 定义 Tokenizer 类
我们将实现一个简单的字符级 Tokenizer，它包括以下功能：

- 构建词汇表（Vocabulary）。
- 将文本转换为索引序列。
- 将索引序列转换回文本。
- 添加特殊标记（如 <PAD>、<UNK>）。

In [ ]:
import torch

class CharTokenizer:
    """
    一个简单的字符级分词器（Tokenizer），用于将文本数据转换为数值化的索引序列。
    """

    def __init__(self, special_tokens=["<PAD>", "<UNK>"]):
        """
        初始化 Tokenizer。

        :param special_tokens: 特殊标记列表，如填充标记（<PAD>）和未知标记（<UNK>）。
        """
        # 初始化特殊标记
        self.special_tokens = special_tokens
        # 构建初始词汇表，将特殊标记加入词汇表
        self.vocab = {token: idx for idx, token in enumerate(special_tokens)}
        # 当前词汇表大小
        self.vocab_size = len(self.special_tokens)
        # 反向映射（索引到标记）暂未初始化
        self.idx_to_token = None

    def fit(self, texts):
        """
        根据提供的文本数据构建词汇表。

        :param texts: 文本列表，用于构建词汇表。
        """
        # 遍历所有文本
        for text in texts:
            # 遍历文本中的每个字符
            for char in text:
                # 如果字符不在当前词汇表中，则添加到词汇表
                if char not in self.vocab:
                    self.vocab[char] = len(self.vocab)
        # 更新词汇表大小
        self.vocab_size = len(self.vocab)
        # 构建反向映射（索引到标记）
        self.idx_to_token = {idx: token for token, idx in self.vocab.items()}

    def encode(self, text, max_length=None, padding=False, truncation=False):
        """
        将文本转换为索引序列。

        :param text: 输入文本。
        :param max_length: 最大序列长度。
        :param padding: 是否填充到最大长度。
        :param truncation: 是否截断超出最大长度的部分。
        :return: 索引序列（Tensor）。
        """
        # 将文本拆分为字符
        tokens = [char for char in text]
        # 将字符映射为索引，未知字符映射为 <UNK> 的索引
        indices = [self.vocab.get(token, self.vocab["<UNK>"]) for token in tokens]

        # 如果指定了最大长度
        if max_length is not None:
            # 如果启用截断，则截断超出最大长度的部分
            if truncation:
                indices = indices[:max_length]
            # 如果启用填充，则将序列填充到最大长度
            if padding:
                indices += [self.vocab["<PAD>"]] * (max_length - len(indices))

        # 将索引列表转换为 PyTorch Tensor
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices):
        """
        将索引序列转换回文本。

        :param indices: 索引序列（Tensor 或列表）。
        :return: 文本。
        """
        # 将索引序列中的每个索引映射回对应的字符
        return "".join(self.idx_to_token[idx.item()] for idx in indices)

    def __len__(self):
        """
        返回词汇表大小。
        """
        return self.vocab_size



### 3. **代码解释**
#### **初始化**
- `special_tokens`：定义了特殊的标记，如 `<PAD>`（填充）和 `<UNK>`（未知字符）。
- `vocab`：词汇表，将字符映射为索引。
- `vocab_size`：词汇表大小。

#### **构建词汇表**
- `fit` 方法：遍历所有文本，将每个字符加入词汇表。如果字符已存在，则跳过。

#### **编码**
- `encode` 方法：
  - 将文本拆分为字符。
  - 将字符映射为索引，未知字符映射为 `<UNK>` 的索引。
  - 根据 `max_length` 进行填充或截断。

#### **解码**
- `decode` 方法：将索引序列转换回文本。

#### **特殊功能**
- `padding`：将序列填充到指定长度。
- `truncation`：将序列截断到指定长度。

### 4. **扩展**
上述实现是一个简单的字符级 Tokenizer。你可以根据需要扩展它，例如：
- 支持子词分词（如 BPE 或 WordPiece）。
- 添加更多特殊标记（如 `[CLS]`、`[SEP]`）。
- 支持并行化处理大量文本。

### 5. **总结**
从零开始实现一个 Tokenizer 是一个很好的学习过程。虽然 PyTorch 本身没有内置的 Tokenizer，但你可以通过简单的 Python 代码实现一个基本的分词器。对于更复杂的任务，建议使用现成的库（如 Hugging Face 的 `transformers`），它们提供了高效且功能丰富的 Tokenizer 实现。

## 使用toenizer

In [2]:
# 示例文本数据
texts = ["hello world", "pytorch is great", "tokenizer example"]

# 初始化 Tokenizer
tokenizer = CharTokenizer()

# 构建词汇表
tokenizer.fit(texts)

# 打印词汇表大小
print(f"Vocabulary Size: {len(tokenizer)}")

# 编码示例
encoded = tokenizer.encode("hello pytorch", max_length=20, padding=True, truncation=True)
print(f"Encoded: {encoded}")

# 解码示例
decoded = tokenizer.decode(encoded)
print(f"Decoded: {decoded}")

Vocabulary Size: 23
Encoded: tensor([ 2,  3,  4,  4,  5,  6, 10, 11, 12,  5,  8, 13,  2,  0,  0,  0,  0,  0,
         0,  0])
Decoded: hello pytorch<PAD><PAD><PAD><PAD><PAD><PAD><PAD>
